In [1]:
!pip install flask pyngrok --quiet
!curl -fsSL https://ollama.com/install.sh | sh

import threading
import subprocess
import time
from flask import Flask, request, jsonify, Response
import requests
from pyngrok import ngrok


# 1. Lanzar Ollama
def run_ollama():
    subprocess.Popen(["ollama", "serve"])

threading.Thread(target=run_ollama).start()

# Esperamos a que Ollama inicie
time.sleep(5)

subprocess.run(["ollama", "pull", "llama3.2"])

# 2. Proxy Flask
app = Flask(__name__)

OLLAMA_API = "http://127.0.0.1:11434"

# Endpoint para generar texto
@app.route("/generate", methods=["POST"])
def generate():
    try:
        r = requests.post(f"{OLLAMA_API}/api/generate", json=request.json, stream=True)
        return Response(r.iter_lines(), content_type="text/plain")
    except Exception as e:
        return jsonify({"error": str(e)}), 500

# Listar modelos (ollama list)
@app.route("/models", methods=["GET"])
def list_models():
    try:
        r = requests.get(f"{OLLAMA_API}/api/tags")
        return jsonify(r.json())
    except Exception as e:
        return jsonify({"error": str(e)}), 500

# Descargar modelo (ollama pull)
@app.route("/pull", methods=["POST"])
def pull_model():
    try:
        r = requests.post(f"{OLLAMA_API}/api/pull", json=request.json, stream=True)
        return Response(r.iter_lines(), content_type="text/plain")
    except Exception as e:
        return jsonify({"error": str(e)}), 500

# Eliminar modelo (ollama delete)
@app.route("/delete", methods=["POST"])
def delete_model():
    try:
        r = requests.post(f"{OLLAMA_API}/api/delete", json=request.json)
        return jsonify(r.json())
    except Exception as e:
        return jsonify({"error": str(e)}), 500

# Crear modelo personalizado (ollama create)
@app.route("/create", methods=["POST"])
def create_model():
    try:
        r = requests.post(f"{OLLAMA_API}/api/create", json=request.json)
        return jsonify(r.json())
    except Exception as e:
        return jsonify({"error": str(e)}), 500

# Mostrar detalles del modelo (ollama show)
@app.route("/show", methods=["POST"])
def show_model():
    try:
        r = requests.post(f"{OLLAMA_API}/api/show", json=request.json)
        return jsonify(r.json())
    except Exception as e:
        return jsonify({"error": str(e)}), 500

def run_flask():
    app.run(host="0.0.0.0", port=5000)

threading.Thread(target=run_flask).start()

# 3. Ngrok
ngrok.set_auth_token("2x0nTZ8cIsu86Gjyhs325SFCw3p_4HgiJcdarFsp6s3yzyHDe")
public_url = ngrok.connect(5000)
print("Tu endpoint público es:", public_url)


>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit


Tu endpoint público es: NgrokTunnel: "https://5cf0-34-125-180-56.ngrok-free.app" -> "http://localhost:5000"
